Today we will fit two versions of the Bradley-Terry model using the current season of NFL data. We will fit the linear model for score differential, and we fit the logistic model for win-loss outcomes. In the exercises, we will use both of these models to predict the outcomes of future games.

In [2]:
install.packages("nflreadr")                                                                      #

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [3]:
data <- nflreadr::load_schedules(seasons = 2023) |>                                               #
  dplyr::filter(game_type == "REG")   # filter to regular season games                            #

We start with some data wrangling. First we construct the matrix X that holds the +1/-1/0 in each entry to keep track of who is the home and away team in each game.

In [4]:
# Set up the regression matrix that encodes home team as +1, away team as -1                      #
team_matrix_home <- model.matrix(~ home_team, data = data)  # this matrix encodes home team as +1 #
team_matrix_away <- model.matrix(~ away_team, data = data)  # this matrix encodes away team as +1 #
team_matrix <- team_matrix_home - team_matrix_away          # +1 for home team; -1 for away team  #

# Convert regression matrix to dataframe                                                          #
team_data <- as.data.frame(team_matrix[, -1])         # drop first column (intercept)             #
names(team_data) <- sort(unique(data$home_team))[-1]  # attach clean team names to team_data      #

To estimate the linear Bradley-Terry model for score differential, we use the `lm` function. Note that the `result` column in `data` holds the score differential from the perspective of the home team.

In [5]:
linear_model <- lm(data$result ~ ., data = team_data)                                             #
summary(linear_model)                                                                             #


Call:
lm(formula = data$result ~ ., data = team_data)

Residuals:
    Min      1Q  Median      3Q     Max 
-26.639  -8.408  -0.382   6.976  39.185 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)   2.6071     0.7802   3.342 0.000965 ***
ATL           0.4538     4.4901   0.101 0.919585    
BAL          18.6192     4.3570   4.273 2.78e-05 ***
BUF          11.9193     4.6170   2.582 0.010429 *  
CAR          -4.4437     4.5919  -0.968 0.334156    
CHI           3.4066     4.5070   0.756 0.450476    
CIN           7.1538     4.3570   1.642 0.101913    
CLE           8.7732     4.3504   2.017 0.044848 *  
DAL          14.6198     4.3818   3.336 0.000983 ***
DEN           1.6497     4.6512   0.355 0.723137    
DET           9.7025     4.6082   2.106 0.036287 *  
GB            6.6199     4.6082   1.437 0.152144    
HOU           5.8818     4.4543   1.320 0.187931    
IND           4.1069     4.5567   0.901 0.368339    
JAX           6.8801     4.5567   1.510 0

To estimate the logistic Bradley-Terry model for win-loss outcomes, we use the `glm` function.

In [6]:
logistic_model <- glm(data$result > 0 ~ ., data = team_data, family = binomial())                 #
summary(logistic_model)                                                                           #


Call:
glm(formula = data$result > 0 ~ ., family = binomial(), data = team_data)

Coefficients:
            Estimate Std. Error z value Pr(>|z|)   
(Intercept)  0.25379    0.13866   1.830  0.06720 . 
ATL          0.04142    0.81615   0.051  0.95952   
BAL          2.57980    0.86282   2.990  0.00279 **
BUF          1.32628    0.85249   1.556  0.11976   
CAR         -1.29660    1.01249  -1.281  0.20033   
CHI          0.17366    0.82183   0.211  0.83265   
CIN          1.56690    0.80590   1.944  0.05186 . 
CLE          1.98661    0.83071   2.391  0.01678 * 
DAL          1.68969    0.83822   2.016  0.04382 * 
DEN          0.54752    0.85160   0.643  0.52027   
DET          1.66289    0.87275   1.905  0.05673 . 
GB           0.76718    0.83649   0.917  0.35906   
HOU          1.22160    0.82868   1.474  0.14044   
IND          1.02750    0.84036   1.223  0.22145   
JAX          1.22995    0.83900   1.466  0.14265   
KC           1.33086    0.84868   1.568  0.11684   
LA           1.64599

EXERCISE #0

According to the model we've estimated, what is the predicted score differential of Sunday Night Football this weekend (Bears @ Texans)?

Note: Admittedly, we are using 2023 data to predict a game in 2024, but the idea is to practice the mechanics of it.

In [14]:
coef(linear_model)["(Intercept)"] + coef(linear_model)["HOU"] - coef(linear_model)["CHI"]         #

(Intercept) 
   5.082365

EXERCISE #1

If (hypothetically) the Houston Texans were to play a game at home against the Arizona Cardinals, what would be the predicted score differential?

In [15]:
coef(linear_model)["(Intercept)"] + coef(linear_model)["HOU"] - 0                                 #

(Intercept) 
   8.488987

EXERCISE #2

What is the estimated *probability* that the Texans beat the Bears on Sunday Night Football this weekend?

In [16]:
eta <- coef(logistic_model)["(Intercept)"] + coef(logistic_model)["HOU"] - coef(logistic_model)["CHI"]
exp(eta) / (1 + exp(eta))                                                                         #

(Intercept) 
  0.7861272

EXERCISE #3

If (hypothetically) the Houston Texans were to play a game at home against the Arizona Cardinals, what is the estimated probability that Houston would win?

In [17]:
eta <- coef(logistic_model)["(Intercept)"] + coef(logistic_model)["HOU"] - 0                      #
exp(eta) / (1 + exp(eta))                                                                         #

(Intercept) 
   0.813876

EXERCISE #4 (Challenge)

Suppose you wanted to predict win-loss outcome probabilities but you wanted to leverage score differentials (because we've learned that score differential is more predictive of future record than are win-loss outcomes). Can you think of a way to extract win-loss probability predictions from the score differential Bradley-Terry model?